[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_dices/observation_prediction_dices.ipynb)

# Transduction from observation to prediction for dices

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_dices.sh http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.sh
! curl -o observation_prediction_dices.py http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.py
! curl -o __init__.py http://josd.github.io/eye/transduction/transduction_dices/__init__.py
! curl -o sample_dices.observation http://josd.github.io/eye/transduction/transduction_dices/sample_dices.observation
! chmod +x observation_prediction_dices.sh
% rm -fr ~/t2t_data/observation_prediction_dices
% rm -fr ~/t2t_train/observation_prediction_dices/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.2)
Requirement not upgraded as not directly required: requests in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.18.4)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.98)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.6.7)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.11.0)
Requirement not upgraded as not directly required: scipy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.19.1)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.6/dist-packages (from tens

In [2]:
# See the observation_prediction_dices problem

! pygmentize -g observation_prediction_dices.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionDices(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for dices."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      outcome = random.randint(1, 6)
      yield {
    

In [3]:
# See the observation_prediction_dices script

! pygmentize -g observation_prediction_dices.sh

#!/bin/bash
PROBLEM=observation_prediction_dices
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=1000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_dices.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_dices.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM 

In [4]:
# Run the observation_prediction_dices script

! ./observation_prediction_dices.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_dices
INFO:tensorflow:Generating data for observation_prediction_dices.
INFO:tensorflow:Generating vocab file: /content/t2t_data/observation_prediction_dices/vocab.observation_prediction_dices.16384.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 249
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 144
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 144
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 144
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iteration 0

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6e41fa3a58>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 1234, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
allow_soft_placement: true
graph_options {
  optimizer_options {
  }
}
, '_save_checkpoints_steps': 100, '_kee

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_9134_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_9134_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_9134_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-22:50:46
INFO:tensorflow:Graph was finalized.
2018-05-14 22:50:46.565268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-14 22:50:46.565380: I tensorflow/core/c

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_9134_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_9134_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_9134_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-22:51:39
INFO:tensorflow:Graph was finalized.
2018-05-14 22:51:39.978074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-14 22:51:39.978217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:91

INFO:tensorflow:Transforming 'targets' with symbol_modality_9134_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_9134_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-22:52:32
INFO:tensorflow:Graph was finalized.
2018-05-14 22:52:32.750688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-14 22:52:32.750797: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-14 22:52:32.750831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-14 22:52:32.750860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-14 22:52:32.751052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, 

...

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-22:55:14
INFO:tensorflow:Saving dict for global step 600: global_step = 600, loss = 0.5617387, metrics-observation_prediction_dices/accuracy = 0.72055554, metrics-observation_prediction_dices/accuracy_per_sequence = 0.16166666, metrics-observation_prediction_dices/accuracy_top5 = 0.9423016, metrics-observation_prediction_dices/approx_bleu_score = 0.36995107, metrics-observation_prediction_dices/neg_log_perplexity = -0.6624137, metrics-observation_prediction_dices/rouge_2_fscore = 0.16166668, metrics-observation_predictio

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-22:56:08
INFO:tensorflow:Saving dict for global step 700: global_step = 700, loss = 0.55440366, metrics-observation_prediction_dices/accuracy = 0.72063494, metrics-observation_prediction_dices/accuracy_per_sequence = 0.16190477, metrics-observation_prediction_dices/accuracy_top5 = 0.9437302, metrics-observation_prediction_dices/approx_bleu_score = 0.37013835, metrics-observation_prediction_dices/neg_log_perplexity = -0.6652599, metrics-observation_prediction_dices/rouge_2_fscore = 0.16190477, metrics-observation_prediction_dices/rouge_L_fscore = 0.7206349
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_dices/observation_prediction_dices-train*
INFO:tensorflow

INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-22:57:01
INFO:tensorflow:Saving dict for global step 800: global_step = 800, loss = 0.5496104, metrics-observation_prediction_dices/accuracy = 0.7221429, metrics-observation_prediction_dices/accuracy_per_sequence = 0.16642857, metrics-observation_prediction_dices/accuracy_top5 = 0.94277775, metrics-observation_prediction_dices/approx_bleu_score = 0.37573272, metrics-observation_prediction_dices/neg_log_perplexity = -0.6677228, metrics-observation_prediction_dices/rouge_2_fscore = 0.16642858, metrics-observation_prediction_dices/rouge_L_fscore = 0.7221428
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_dices/observation_prediction_dices-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INF

INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-22:57:55
INFO:tensorflow:Saving dict for global step 900: global_step = 900, loss = 0.546945, metrics-observation_prediction_dices/accuracy = 0.7215873, metrics-observation_prediction_dices/accuracy_per_sequence = 0.1647619, metrics-observation_prediction_dices/accuracy_top5 = 0.94222224, metrics-observation_prediction_dices/approx_bleu_score = 0.37368163, metrics-observation_prediction_dices/neg_log_perplexity = -0.67135036, metrics-observation_prediction_dices/rouge_2_fscore = 0.16476192, metrics-observation_prediction_dices/rouge_L_fscore = 0.7215873
INFO:tensorflow:Training model for 100 steps
INFO:tensorflow:Reading data files from /content/t2t_data/observation_prediction_dices/observation_prediction_dices-train*
INFO:tensorflow:partition: 0 num_data_files: 7
INFO

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-05-14-22:58:48
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.54491967, metrics-observation_prediction_dices/accuracy = 0.72373015, metrics-observation_prediction_dices/accuracy_per_sequence = 0.17119047, metrics-observation_prediction_dices/accuracy_top5 = 0.9411111, metrics-observation_prediction_dices/approx_bleu_score = 0.38138908, metrics-observation_prediction_dices/neg_log_perplexity = -0.6742336, metrics-observation_prediction_dices/rouge_2_fscore = 0.17119049, metrics-observation_prediction_dices/rouge_L_fscore = 0.7237301
INFO:tensorflow:Stop training model as max steps reached
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype f

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_dices.observation
print("->-")
! pygmentize -g sample_dices.prediction

A_THROW 100001
A_THROW 100002
A_THROW 100003
A_THROW 100004
A_THROW 100005
A_THROW 100006
->-
 1	-1.60	 5	-1.62	 4	-1.66
 5	-1.61	 1	-1.62	 4	-1.67
 1	-1.61	 5	-1.61	 4	-1.68
 1	-1.61	 5	-1.63	 4	-1.67
 1	-1.62	 5	-1.63	 4	-1.68
 1	-1.61	 5	-1.62	 4	-1.68
